In [0]:
import pandas as pd
import plotly.graph_objects as go

# Lê os dados
df = spark.sql("SELECT * FROM classes.gold.df_exames").toPandas()

df['data_realizacao'] = pd.to_datetime(df['data_realizacao'])
df['ano_mes'] = df['data_realizacao'].dt.to_period('M')

# Agrupa por mês
monthly = (df.groupby('ano_mes')
           .size()
           .reset_index(name='exames')
           .sort_values('ano_mes'))
monthly['ano_mes'] = monthly['ano_mes'].astype(str)

# Gráfico lindo
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly['ano_mes'],
    y=monthly['exames'],
    mode='lines+markers',
    line=dict(color='#1E88E5', width=5),
    marker=dict(size=10, color='#1565C0', line=dict(width=2, color='white')),
    hovertemplate='<b>%{x}</b><br>Exames realizados: <b>%{y}</b><extra></extra>',
    name='Exames mensais'
))

# Área preenchida suave
fig.add_trace(go.Scatter(
    x=monthly['ano_mes'].tolist() + monthly['ano_mes'].tolist()[::-1],
    y=monthly['exames'].tolist() + [0]*len(monthly),
    fill='toself',
    fillcolor='rgba(30, 136, 229, 0.15)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

fig.update_layout(
    title=dict(
        text="<b>Evolução do Volume de Exames Médicos</b><br><sup>Período: 2020 – 2025</sup>",
        x=0.5,
        font=dict(size=22, color="#1E3A5F")
    ),
    xaxis=dict(
        title="",
        tickangle=45,
        showgrid=False,
        tickfont=dict(size=12),
        titlefont=dict(size=14)
    ),
    yaxis=dict(
        title="Número de Exames",
        showgrid=True,
        gridcolor='rgba(200,200,200,0.3)',
        tickfont=dict(size=12),
        titlefont=dict(size=14)
    ),
    height=650,
    plot_bgcolor='white',
    paper_bgcolor='white',
    hoverlabel=dict(bgcolor="white", font_size=14),
    margin=dict(l=70, r=50, t=110, b=80),
    font=dict(family="Segoe UI, sans-serif", color="#2d3436")
)

fig.show()